In [1]:
# Importa as bibliotecas e arquivos necessarios

import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas
import os
from network_architectures import dense_autoencoder

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

%load_ext tensorboard
!rm -rf ./logs/ 

In [2]:
#   Faz o download do fashion_mnist da base de dados do keras
mnist = keras.datasets.mnist

#   Carrega esse dataset, ja separando entre dados de treinamento e dados de teste
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#   Podemos verificar o formato desses arquivos
train_images.shape
test_images.shape
train_labels.shape
test_labels.shape

#   Segunda parte - processar os dados -----------------------------------------
#   Em seguida, precisamos processar esses dados. Nesse caso, o processamento se resume a
#   limitar o valor de cada pixel das imagens no intervalo [0,1]. Fazemos isso dividindo por 255
train_images = train_images / 255.0
test_images = test_images / 255.0

In [3]:
#   Terceira parte - Definir o modelo e fazer o treinamento --------------------

#   Carregamos a arquitetura do nosso modelo do nosso arquivo de arquiteturas. Isso deixa o codigo
#   bem modular e limpo
model = dense_autoencoder()

In [4]:
#   Compilamos o nosso modelo. Compilar significa dizer qual a funcao custo e otimizador vamos utilizar (entre outras coisinhas)

#   Nesse caso, o otimizador eh o "Adam". A funcao custo eh uma funcao mse
model.compile(optimizer='adam',
              loss='mse')

In [5]:
#Resumo do modelo
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                50240     
_________________________________________________________________
dense_1 (Dense)              (None, 784)               50960     
_________________________________________________________________
reshape (Reshape)            (None, 28, 28)            0         
Total params: 101,200
Trainable params: 101,200
Non-trainable params: 0
_________________________________________________________________


In [6]:
#   Dessa vez, antes de proseguirmos com o treinamento, vamos criar um callback.
#   O callback serve para que o keras faca coisas entre epocas, como por exemplo,
#   salvar o melhor modelo. Faremos um callback para que ele salve a melhor epoca,
#   e mude o "learning rate" dependendo da epoca

In [7]:
#   um learning rate scheduler muda o learning rate de acordo com qual a epoca atual
#   nosso learning scheduler simplesmente muda a taxa ao passar de 10 epocas

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.01,
    decay_steps=10,
    decay_rate=0.9,
    staircase=True)

learning_schedule = keras.callbacks.LearningRateScheduler(lr_schedule)

In [8]:
# Este callback faz com que o treinamento acabe mais rapidamente, caso a função custo não esteja mais sendo minimizada

earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [9]:
#   Para salvarmos a melhor epoca, usamos o callback de ModelCheckpoint
model_checkpoint = keras.callbacks.ModelCheckpoint('model_checkpoint.hdf5',
                    monitor='val_loss',
                    verbose=2,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='auto')

In [10]:
#   Para vizualizarmos melhor o treinamento do modelo, usaremos o Tensorboard
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [11]:
#   criamos o callback para passar para o metodo .fit
callback = [learning_schedule, model_checkpoint, tensorboard, earlystop]

In [12]:
#   Agora vamos enfim treinar o modelo. Usamos o metodo "fit", passando como parametro nossos dados de treino, teste, e por quantas epocas
#   Esse metodo retorna os dados mostrados durante o treinamento, e em geral e interessante salva-los.

fit_history = model.fit(train_images, train_images, validation_data=(test_images, test_images), epochs=100, batch_size=64, callbacks=callback)

Epoch 1/100
938/938 [==============================] - 3s 3ms/step - loss: 0.0352 - val_loss: 0.0116

Epoch 00001: val_loss improved from inf to 0.01160, saving model to model_checkpoint.hdf5
Epoch 2/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0121 - val_loss: 0.0113

Epoch 00002: val_loss improved from 0.01160 to 0.01129, saving model to model_checkpoint.hdf5
Epoch 3/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0117 - val_loss: 0.0110

Epoch 00003: val_loss improved from 0.01129 to 0.01103, saving model to model_checkpoint.hdf5
Epoch 4/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0115 - val_loss: 0.0108

Epoch 00004: val_loss improved from 0.01103 to 0.01082, saving model to model_checkpoint.hdf5
Epoch 5/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0114 - val_loss: 0.0109

Epoch 00005: val_loss did not improve from 0.01082
Epoch 6/100
938/938 [==============================] - 2s 2ms/step 

938/938 [==============================] - 2s 2ms/step - loss: 0.0096 - val_loss: 0.0093

Epoch 00048: val_loss did not improve from 0.00925
Epoch 49/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0095 - val_loss: 0.0094

Epoch 00049: val_loss did not improve from 0.00925
Epoch 50/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0096 - val_loss: 0.0093

Epoch 00050: val_loss did not improve from 0.00925
Epoch 51/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0094 - val_loss: 0.0093

Epoch 00051: val_loss did not improve from 0.00925
Epoch 52/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0095 - val_loss: 0.0092

Epoch 00052: val_loss improved from 0.00925 to 0.00917, saving model to model_checkpoint.hdf5
Epoch 53/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0094 - val_loss: 0.0092

Epoch 00053: val_loss did not improve from 0.00917
Epoch 54/100
938/938 [========================

938/938 [==============================] - 2s 2ms/step - loss: 0.0091 - val_loss: 0.0087

Epoch 00098: val_loss improved from 0.00874 to 0.00870, saving model to model_checkpoint.hdf5
Epoch 99/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0090 - val_loss: 0.0087

Epoch 00099: val_loss did not improve from 0.00870
Epoch 100/100
938/938 [==============================] - 2s 2ms/step - loss: 0.0090 - val_loss: 0.0087

Epoch 00100: val_loss improved from 0.00870 to 0.00870, saving model to model_checkpoint.hdf5


In [13]:
# Voce pode vizualizar diversor gráficos e estatísticas sobre o treinamento usando o tensorboard

%tensorboard --logdir logs

In [14]:
#   Quarta parte - Salvar o modelo e outros dados --------------------
#   Neste ponto, nosso modelo ja esta treinado. Em geral, so queremos treinar o modelo uma unica vez, pois demora muito.
#   Vamos entao salvar esse modelo, junto com os dados de treinamento, no disco. Assim, se precisarmos usar ele novamente,
#   so precisamos carrega-lo.

In [15]:
#   Vamos primeiro salvar o historico de treinamento. Para isso, criaremos um dataframe no pandas (por ser mais facil de trabalhar assim)
fit_history_df = pandas.DataFrame(fit_history.history)

#   Com isso podemos salvar esses dados no disco diretamente
with open('fit_history.csv', mode='w') as f:
    fit_history_df.to_csv(f)
#   Reparar que um arquivo csv foi salvo no disco. Da uma olhadinha nesse arquivo depois

In [16]:
#   Em seguida, vamos salvar o modelo. Para isso, salvamos tanto a arquitetura quanto os pesos calculados.
#   A arquitetura do modelo podemos salvar como um arquivo json, com a funcao
model_json_string = model.to_json()
open('architecture.json', 'w').write(model_json_string)

1422

In [17]:
#   Eh importante notar que estamos salvando a ULTIMA EPOCA do modelo, e que ela nao necessariamente sera a melhor.
#   Para salvarmos a melhor epoca do modelo, precisamos configurar um "callback" no keras. Faremos isso em outro tutorial

#   Agora um arquivo json foi salvo. Da uma olhadinha nesse arquivo tbm
#   Ta na hora de salvar os pesos. Isso eh feito com a funcao
model.save_weights('model_weights.h5', overwrite=True)

In [18]:
#   Agora esse arquivo .h5 tbm foi salvo no disco. Com isso temos todo o nosso modelo salvo no disco, e nao precisaremos treina-lo novamente
#   Nosso arquivo de treinamento acaba por aqui. Agora vamos analisar o modelo no outro arquivo